## AutoGluon

In this notebook, we use __AutoGluon__ to predict the __Outcome Type__ field of our review dataset.


[AutoGluon](https://autogluon.mxnet.io/tutorials/tabular_prediction/index.html) implements many of the best practices that we have discussed in this class, and more!  In particular, it sets itself apart from other AutoML solutions by having excellent automated feature engineering that can handle text data and missing values without any hand-coded solutions (see their [paper](https://arxiv.org/abs/2003.06505) for details).  It is too new to be in an existing Sagemaker kernel, so let's install it.

1. <a href="#1">Set up AutoGluon</a>
2. <a href="#2">Read the datasets</a>
3. <a href="#3">Train a classifier with AutoGluon</a>
4. <a href="#4">Model evaluation</a>
5. <a href="#5">Clean up model artifacts</a>

__Austin Animal Center Dataset__:

In this exercise, we are working with pet adoption data from __Austin Animal Center__. We have two datasets that cover intake and outcome of animals. Intake data is available from [here](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm) and outcome is from [here](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Outcomes/9t4d-g238). 

In order to work with a single table, we joined the intake and outcome tables using the "Animal ID" column and created a single __review.csv__ file. We also didn't consider animals with multiple entries to the facility to keep our dataset simple. If you want to see the original datasets and the merged data with multiple entries, they are available under `DATA/review` folder: Austin_Animal_Center_Intakes.csv, Austin_Animal_Center_Outcomes.csv and Austin_Animal_Center_Intakes_Outcomes.csv.

__Dataset schema:__ 
- __Pet ID__ - Unique ID of pet
- __Outcome Type__ - State of pet at the time of recording the outcome (0 = not placed, 1 = placed). This is the field to predict.
- __Sex upon Outcome__ - Sex of pet at outcome
- __Name__ - Name of pet 
- __Found Location__ - Found location of pet before entered the center
- __Intake Type__ - Circumstances bringing the pet to the center
- __Intake Condition__ - Health condition of pet when entered the center
- __Pet Type__ - Type of pet
- __Sex upon Intake__ - Sex of pet when entered the center
- __Breed__ - Breed of pet 
- __Color__ - Color of pet 
- __Age upon Intake Days__ - Age of pet when entered the center (days)
- __Age upon Outcome Days__ - Age of pet at outcome (days))

## 1. <a name="1">Set up AutoGluon</a>
(<a href="#0">Go to top</a>)

In [1]:
!pip install --upgrade pip
!pip install --upgrade mxnet autogluon

In [15]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    warnings.filterwarnings("ignore", category=DeprecationWarning)

## 2. <a name="2">Read the dataset</a>
(<a href="#0">Go to top</a>)

Let's read the dataset into a dataframe, using Pandas, and split the dataset into train and test sets (AutoGluon will handle the validation itself).

In [16]:
import pandas as pd

df = pd.read_csv('../../DATA/review/review_dataset.csv')

In [17]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.1, shuffle=True, random_state=23)

## 3. <a name="3">Train a classifier with AutoGluon</a>
(<a href="#0">Go to top</a>)

We can run AutoGluon with a short snippet.

In [18]:
from autogluon.tabular import TabularPredictor

k = 100 # grab less data for a quick demo
#k = train_data.shape[0] # grad the whole dataset; 

predictor = TabularPredictor(label='Outcome Type').fit(train_data=train_data.head(k))

No path specified. Models will be saved in: "AutogluonModels/ag-20210521_070155/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20210521_070155/"
AutoGluon Version:  0.2.0
Train Data Rows:    100
Train Data Columns: 12
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1.0, 0.0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    6163.98 MB
	Train Data (Original)  Memory Usage: 0.07 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes

## 4. <a name="4">Model evaluation</a>
(<a href="#0">Go to top</a>)

In [20]:
y_pred = predictor.predict(test_data.head(k))
predictor.evaluate_predictions(y_true=test_data['Outcome Type'].head(k), y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.73
Evaluations on test data:
{
    "accuracy": 0.73,
    "balanced_accuracy": 0.7003263973888209,
    "mcc": 0.45220701825084986,
    "f1": 0.7938931297709922,
    "precision": 0.7027027027027027,
    "recall": 0.9122807017543859
}


{'accuracy': 0.73,
 'balanced_accuracy': 0.7003263973888209,
 'mcc': 0.45220701825084986,
 'f1': 0.7938931297709922,
 'precision': 0.7027027027027027,
 'recall': 0.9122807017543859}

## 5. <a name="5">Clean up model artifacts</a>
(<a href="#0">Go to top</a>)

In [12]:
!rm -r AutogluonModels
!rm -r catboost_info
!rm -r dask-worker-space